# 关键概念

欢迎来到``brainscale``的世界！

``brainscale``是一个支持动力学神经网络模型的在线学习Python库。在线学习（online learning）是一种学习范式，它允许模型在不断地接收新数据的同时，不断地更新自己的参数。这种学习方式在许多现实世界的应用中非常有用，比如在机器人控制、智能体的决策制定、以及大规模数据流的处理中。

在这个章节，我将会介绍一些关键概念，这些概念是理解和使用``brainscale``在线学习的基础。这些概念包括：

- 如何构建支持在线学习的高层次神经网络模型。
- 用于定制化网络模块的 模型状态``ETraceState``、模型参数``ETraceParam``和 模型交互算子``ETraceOp``。
- 在线学习算法 ``ETraceAlgorithm``。

``brainscale``精密地整合在以``brainstate``为中心的[脑动力学编程生态系统](https://ecosystem-for-brain-dynamics.readthedocs.io/)中。我们强烈建议您首先熟悉[``brainstate``的基本用法](https://brainstate.readthedocs.io/)，以此能帮助您更好地理解``brainscale``的工作原理。

In [1]:
import brainstate
import brainscale
import brainunit as u

## 1. ``brainscale``支持的动力学模型

``brainscale``并不支持所有类型的动力学模型在线学习。它当前支持的动力学模型具有特定的结构，如下图所示，其中“动力学（dynamics）”与“动力学之间的交互（interaction）”是严格分开的。这种结构的模型可以分解为两个主要部分：

- **动力学部分**：这一部分描述了神经元内在的动力学行为，例如 LIF 神经元模型、FitzHugh-Nagumo 模型以及长短期记忆网络（LSTM）。动力学状态（hidden states）的更新是严格逐元素运算（element-wise operations），但模型可以包含多个动力学状态。
- **交互部分**：这一部分描述了神经元之间的交互，例如权重矩阵和连接矩阵。模型动力学之间的交互可以通过标准的矩阵乘法、卷积操作或稀疏操作等方式实现。


![](../_static/model-dynamics-supported.png)



让我们通过一个简单的网络模型示例来阐明``brainscale``所支持的动力学模型。我们考虑一个基本的LIF神经元网络，其动力学由以下微分方程描述：

$$
\begin{aligned}
\tau \frac{dv_i}{dt} &= -v_i + I_{\text{ext}} + s_i \\
\tau_s \frac{ds_i}{dt} &= -s_i + \sum_{j} w_{ij} \delta(t - t_j)
\end{aligned}
$$

其中，$v_i$是神经元的膜电位，当神经元膜电位超过阈值$v_{th}$时，神经元会发放动作电位，并且膜电位会被重置为$v_{\text{reset}}$。

$$
\begin{aligned}
z_i & = \mathcal{H}(v_i-v_{th}) \\
v_i & \leftarrow v_{\text{reset}}
\end{aligned}
$$

另外，$s_i$是突触后电流，$I_{\text{ext}}$是外部输入电流，$w_{ij}$是神经元$i$到神经元$j$的突触权重，$\delta(t - t_j)$是Dirac函数，表示在时间$t_j$接收到一个突触事件。$\tau$和$\tau_s$分别是膜电位和突触后电流的时间常数。

通过数值积分的方法，我们对上述微分方程进行离散化，并将其写成向量形式，从而得到以下动力学更新规则：

$$
\begin{aligned}
\mathbf{v}_i^{t+1} &= \mathbf{v}_i^{t} + \frac{\Delta t}{\tau} (-\mathbf{v}_i^{t} + \mathbf{I}_{\text{ext}} + \mathbf{s}^t) \\
\mathbf{s}_i^{t+1} &= \mathbf{s}_i^{t} + \frac{\Delta t}{\tau_s} (-\mathbf{s}_i^{t} + \underbrace{  W \mathbf{z}^t  } _ {\text{neuronal interaction}} )
\end{aligned}
$$

可以看到，LIF神经元的动力学更新是逐元素进行的，而交互部分的更新则通过矩阵乘法实现。所有`brainscale`支持的动力学模型均可分解为这样的动力学部分和交互部分。值得注意的是，绝大多数循环神经网络模型都符合这一结构，因此`brainscale`能够支持大部分循环神经网络模型的在线学习。


## 2. ``brainscale.nn``：构建支持在线学习的神经网络

在``brainscale``中，我们可以使用与``brainstate``完全相同的语法来构建支持在线学习的神经网络模型。有关详细教程，请参考 [构建人工神经网络](https://brainstate.readthedocs.io/en/latest/tutorials/artificial_neural_networks-zh.html) 和 [构建脉冲神经网络](https://brainstate.readthedocs.io/en/latest/tutorials/spiking_neural_networks-zh.html)。

唯一的区别在于，我们需要使用[``brainscale.nn``模块](../apis/nn.rst)中的组件来构建神经网络模型。这些组件是``brainstate.nn``的扩展，专门设计用于支持在线学习的单元模块。

以下是一个简单示例，展示如何使用``brainscale.nn``模块构建一个基本的LIF神经元网络。

In [2]:
class LIF_Delta_Net(brainstate.nn.Module):
    def __init__(
        self,
        n_in,
        n_rec,
        tau_mem=5. * u.ms,
        V_th=1. * u.mV,
        spk_fun=brainstate.surrogate.ReluGrad(),
        spk_reset: str = 'soft',
        rec_scale: float = 1.,
        ff_scale: float = 1.,
    ):
        super().__init__()

        # 使用 brainscale.nn 内的 LIF 模型
        self.neu = brainscale.nn.LIF(n_rec, tau=tau_mem, spk_fun=spk_fun, spk_reset=spk_reset, V_th=V_th)

        # 构建输入和循环连接权重
        rec_init = brainstate.init.KaimingNormal(rec_scale, unit=u.mV)
        ff_init = brainstate.init.KaimingNormal(ff_scale, unit=u.mV)
        w_init = u.math.concatenate([ff_init([n_in, n_rec]), rec_init([n_rec, n_rec])], axis=0)

        # 使用 delta 突触投射來构建输入和循环连接
        self.syn = brainstate.nn.DeltaProj(
            # 使用 brainscale.nn 内的 Linear 模型
            comm=brainscale.nn.Linear(n_in + n_rec, n_rec, w_init=w_init, b_init=brainstate.init.ZeroInit(unit=u.mV)),
            post=self.neu
        )

    def update(self, spk):
        inp = u.math.concatenate([spk, self.neu.get_spike()], axis=-1)
        self.syn(inp)
        self.neu()
        return self.neu.get_spike()

在这个示例中，我们定义了一个``LIF_Delta_Net``类，该类继承自``brainstate.nn.Module``。该类包含一个LIF神经元模型``self.neu``以及一个用于输入和循环连接的``DeltaProj``模块``self.syn``。

接下来，我们将构建一个三层的门控循环单元（GRU）神经网络模型：

In [3]:
class GRU_Net(brainstate.nn.Module):
    def __init__(
        self,
        n_in: int,
        n_rec: int,
        n_out: int,
        n_layer: int,
    ):
        super().__init__()

        # 构建 GRU 层
        self.layers = []
        for i in range(n_layer - 1):
            # 使用 brainscale.nn 内的 GRUCell 模型
            self.layers.append(brainscale.nn.GRUCell(n_in, n_rec))
            n_in = n_rec
        self.layers.append(brainscale.nn.GRUCell(n_in, n_out))

    def update(self, x):
        # 更新 GRU 层
        for layer in self.layers:
            x = layer(x)
        return x

可以看到，基于[``brainscale.nn``模块](../apis/nn.rst)构建神经网络模型的过程与基于[``brainstate.nn``模块](https://brainstate.readthedocs.io/en/latest/apis/nn.html)的构建过程完全相同。这意味着，您可以直接利用``brainstate``的教程来构建支持在线学习的神经网络模型。

## 3. ``ETraceState``、``ETraceParam``和``ETraceOp``：定制化网络模块

尽管``brainscale.nn``模块提供了一些基本的网络模块，但它并未涵盖所有可能的网络动力学。因此，我们需要一种机制，允许用户定制网络模块。在``brainscale``中，我们提供了``ETraceState``、``ETraceParam``和``ETraceOp``三个类，供用户进行模块定制。

- **``brainscale.ETraceState``**：代表模块中的模型状态$\mathbf{h}$，用于定义模型的动力学状态，例如LIF神经元的膜电位或指数突触模型的突触后电导。
- **``brainscale.ETraceParam``**：对应模块中的模型参数$\theta$，用于定义模型参数，例如线性矩阵乘法的权重矩阵，还可以用于LIF神经元中自适应学习的时间常数等。所有需要在训练过程中进行梯度更新的参数都应在``ETraceParam``中定义。
- **``brainscale.ETraceOp``**：用于描述输入数据如何基于模型参数计算突触后电流的操作，支持定义模型操作，如线性矩阵乘法、稀疏矩阵乘法和卷积操作。

``ETraceState``、``ETraceParam``和``ETraceOp``是``brainscale``中的三个基本概念，构成了支持在线学习的神经网络模型的基础。

接下来，让我们通过一系列简单示例来说明如何使用``ETraceState``、``ETraceParam``和``ETraceOp``进行网络模块的定制。

### 3.1 ``ETraceState``：模型状态

我们首先考虑一个简单的LIF神经元模型，它的动力学部分由如下的微分方程描述：

$$
\begin{aligned}
\tau \frac{dv_i}{dt} &= -v_i + I_{\text{ext}} + v_\text{rest} \\
z_i & = \mathcal{H}(v_i-v_{th}) \\
v_i & \leftarrow v_{\text{reset}} \quad \text{if} \quad z_i > 0
\end{aligned}
$$

其中，$v_i$是神经元的膜电位，当神经元膜电位超过阈值$v_{th}$时，神经元会发放动作电位，并且膜电位会被重置为$v_{\text{reset}}$。$\mathcal{H}$是一个阶跃函数，表示神经元的发放动作电位，$I_{\text{ext}}$是外部输入电流，$\tau$是膜电位的时间常数，$v_\text{rest}$ 是膜电位的静息电位。

In [4]:
import jax
from typing import Callable


class LIF(brainstate.nn.Neuron):
    """
    Leaky integrate-and-fire neuron model.
    """

    def __init__(
        self,
        size: brainstate.typing.Size,
        R: brainstate.typing.ArrayLike = 1. * u.ohm,
        tau: brainstate.typing.ArrayLike = 5. * u.ms,
        V_th: brainstate.typing.ArrayLike = 1. * u.mV,
        V_reset: brainstate.typing.ArrayLike = 0. * u.mV,
        V_rest: brainstate.typing.ArrayLike = 0. * u.mV,
        V_initializer: Callable = brainstate.init.Constant(0. * u.mV),
        spk_fun: Callable = brainstate.surrogate.ReluGrad(),
        spk_reset: str = 'soft',
        name: str = None,
    ):
        super().__init__(size, name=name, spk_fun=spk_fun, spk_reset=spk_reset)

        # parameters
        self.R = brainstate.init.param(R, self.varshape)
        self.tau = brainstate.init.param(tau, self.varshape)
        self.V_th = brainstate.init.param(V_th, self.varshape)
        self.V_rest = brainstate.init.param(V_rest, self.varshape)
        self.V_reset = brainstate.init.param(V_reset, self.varshape)
        self.V_initializer = V_initializer

    def init_state(self, batch_size: int = None, **kwargs):
        # 这里是最关键的一步，我们定义了一个 ETraceState 类，用于描述膜电位的动力学状态
        self.V = brainscale.ETraceState(brainstate.init.param(self.V_initializer, self.varshape, batch_size))

    def reset_state(self, batch_size: int = None, **kwargs):
        self.V.value = brainstate.init.param(self.V_initializer, self.varshape, batch_size)

    def get_spike(self, V=None):
        V = self.V.value if V is None else V
        v_scaled = (V - self.V_th) / (self.V_th - self.V_reset)
        return self.spk_fun(v_scaled)

    def update(self, x=0. * u.mA):
        last_v = self.V.value
        lst_spk = self.get_spike(last_v)
        V_th = self.V_th if self.spk_reset == 'soft' else jax.lax.stop_gradient(last_v)
        V = last_v - (V_th - self.V_reset) * lst_spk
        # membrane potential
        dv = lambda v: (-v + self.V_rest + self.R * self.sum_current_inputs(x, v)) / self.tau
        V = brainstate.nn.exp_euler_step(dv, V)
        V = self.sum_delta_inputs(V)
        self.V.value = V
        return self.get_spike(V)

在上面的代码中，我们继承``brainstate.nn.Neuron``定义了这个``LIF``模型。这个类包含了一个``ETraceState``类变量``self.V``，用于描述膜电位的动力学状态。在``init_state``方法中，我们初始化了膜电位的动力学状态。在``update``方法中，我们更新了膜电位的动力学状态。实际上，这个类的定义与``brainstate``中的``LIF``类的定义基本上是一模一样的，唯一不同的地方在于``brainstate``使用``brainstate.HiddenState``来描述膜电位的动力学状态，而``brainscale``使用``brainstate.ETraceState``来标记该膜电位的动力学状态是需要用于在线学习的。

因此，我们可以将``brainscale.ETraceState``视为与``brainstate.HiddenState``相对应的概念，专门用于定义需要进行资格迹（eligibility trace）更新的模型状态。如果在程序中将模型状态定义为``brainstate.HiddenState``而非``brainscale.ETraceState``，则``brainscale``的在线学习编译器将无法识别该状态，导致编译后的在线学习规则不再作用于该状态，从而引发模型的梯度更新错误或遗漏。

### 3.2 ``ETraceOp``：模型输入输出函数



``ETraceOp``是另一个描述动力学交互的概念。``ETraceParam``描述了动力学交互中用到的参数，而``ETraceOp``描述了动力学交互的操作。它需要定义为一个具有如下格式要求的函数：

```python
def op(
    x: jax.Array, 
    param: brainscale.ETraceParam
) -> jax.Array:
    pass
```


``ETraceOp``描述了模型输入如何根据参数转换为输出，它可以实现为各种各样的模型的操作，包括线性矩阵乘法、稀疏矩阵乘法、卷积操作等。

首先，我们考虑一个简单的矩阵乘法算子，其计算公式为：

$$
y = W x + b
$$

其中，$W$是权重矩阵，$x$是输入向量，$b$是偏置向量。

针对上面的矩阵乘法算子，我们可以定义如下的``ETraceOp``：

In [6]:
@brainscale.ETraceOp
def matmul(x, w):
    weight = w['weight']
    bias = w['bias']
    return x @ weight + bias

### 3.3 ``ETraceParam`` : 模型参数和操作的组合

首先，对于上面的矩阵乘法算子，我们可以使用``ETraceParam``来定义这些模型参数。

In [ ]:
def generate_weight(
    n_in, n_out, init: Callable = brainstate.init.KaimingNormal()
) -> brainscale.ETraceParam:
    weight = init([n_in, n_out])
    bias = brainstate.init.ZeroInit()([n_out])
    
    # 这里是最关键的一步，我们定义了一个 ETraceParam 类，用于描述权重矩阵和偏置向量
    return brainscale.ETraceParam({'weight': weight, 'bias': bias})

在上述代码中，我们定义了一个``generate_weight``函数，用于生成权重矩阵和偏置向量。该函数返回一个``ETraceParam``对象，以描述权重矩阵和偏置向量。

``brainscale.ETraceParam``是与``brainstate.ParamState``相对应的概念，专门用于定义需要进行资格迹（eligibility trace）更新的模型参数。如果我们在程序中将模型参数$\theta$定义为``brainscale.ETraceParam``，那么``brainscale``的在线学习编译器将对该参数进行具有时序依赖的梯度更新，计算公式为：

$$
\nabla_\theta \mathcal{L}=\sum_{t} \frac{\partial \mathcal{L}^{t}}{\partial \mathbf{h}^{t}} \sum_{k=1}^t \frac{\partial \mathbf{h}^t}{\partial \boldsymbol{\theta}^k},
$$

其中，$\boldsymbol{\theta}^k$是在第$k$时刻使用的权重$\boldsymbol{\theta}$。

相反，如果我们将模型参数$\theta$定义为``brainstate.ParamState``，那么``brainscale``的在线学习编译器仅会计算当前时刻损失函数对权重的偏导值，即：

$$
\nabla_\theta \mathcal{L}=\sum_{t} \frac{\partial \mathcal{L}^{t}}{\partial \mathbf{h}^{t}} \frac{\partial \mathbf{h}^t}{\partial \boldsymbol{\theta}^t}.
$$

这意味着，在``brainscale``的在线学习中，``brainstate.ParamState``被视为不需要进行eligibility trace更新的模型参数，因此失去了对时序依赖信息的梯度计算能力。这样的设计使得模型参数的更新模式更加灵活，从而增加了梯度计算的可定制性。




不仅如此，``ETraceParam``能够用于描述模型参数和操作的组合，包含了模型参数和操作的信息，因此在实例化时需要接收一个参数和一个操作函数。

基于``ETraceParam``，我们可以将上面的矩阵乘法算子定义为：

In [7]:
class Linear(brainstate.nn.Module):
    """
    Linear layer.
    """
    def __init__(
        self,
        in_size: brainstate.typing.Size,
        out_size: brainstate.typing.Size,
        w_init: Callable = brainstate.init.KaimingNormal(),
    ):
        super().__init__()

        # input and output shape
        self.in_size = in_size
        self.out_size = out_size

        # weights
        weight = brainstate.init.param(w_init, [self.in_size[-1], self.out_size[-1]], allow_none=False)
        
        # operation
        op = lambda x, w: u.math.matmul(x, w)
        
        # 这里是最关键的一步，我们定义了一个 ETraceParam 类，用于描述权重矩阵和操作
        self.weight_op = brainscale.ETraceParam(weight, op)

    def update(self, x):
        # ETraceParam 的操作
        return self.weight_op.execute(x)

从上面的代码中可以看到，``ETraceParam``是一个描述模型参数和操作的组合的概念。它包含了模型参数和操作的信息，因此需要接收一个参数和一个操作函数进行实例化。``ETraceParam``的``execute``方法用于执行操作函数，将输入数据转换为输出数据。

至此，我们定义了一个简单的线性层模块``Linear``，它包含了一个权重矩阵和一个矩阵乘法操作。这个模块可以被用于构建支持在线学习的神经网络模型。

## 4. ``ETraceAlgorithm``：在线学习算法

``ETraceAlgorithm``是``brainscale``中的另一个重要概念，它定义了模型的状态更新过程中如何更新eligibility trace，以及定义了模型参数的梯度更新规则。``ETraceAlgorithm``是一个抽象类，专门用于描述``brainscale``内各种形式的在线学习算法。

``brainscale.ETraceAlgorithm``中提供的算法支持，是基于上面提供的``ETraceState``、``ETraceParam``和``ETraceOp``三个基本概念。``brainscale.ETraceAlgorithm``提供了一种灵活的在线学习编译器。它可以支持使用上述三个概念构建的任意神经网络模型的在线学习。


具体来说，目前 ``brainscale`` 支持的在线学习算法有：

- ``brainscale.DiagIODimAlgorithm``：该算法使用 ES-D-RTRL 算法进行在线学习，支持$O(N)$复杂度的在线梯度计算，适用于大规模脉冲神经网络模型的在线学习。具体算法细节可以参考[我们的论文](https://doi.org/10.1101/2024.09.24.614728)。
- ``brainscale.DiagParamDimAlgorithm``：该算法使用 D-RTRL 算法进行在线学习，支持$O(N^2)$复杂度的在线梯度计算，适用于循环神经网络模型和脉冲神经网络模型的在线学习。具体算法细节可以参考[我们的论文](https://doi.org/10.1101/2024.09.24.614728)。
- ``brainscale.DiagHybridDimAlgorithm``：该算法选择性地使用 ES-D-RTRL 或 D-RTRL 算法对模型参数进行在线学习。对于卷积层和高度稀疏连接的层，该算法有更大的倾向使用 D-RTRL 算法进行在线学习，以减少在线学习参数更新所需的计算复杂度。
- 未来，我们将会支持更多的在线学习算法，以满足更多的应用场景。




在下面的例子中，我们将展示如何使用``brainscale.ETraceAlgorithm``来构建一个支持在线学习的神经网络模型。


In [8]:
with brainstate.environ.context(dt=0.1 * u.ms):

    # 定义一个简单的LIF神经元构成的循环神经网络
    model = LIF_Delta_Net(10, 10)
    brainstate.nn.init_all_states(model)
    
    # 将该模型输入到在线学习算法中，以期进行在线学习
    model = brainscale.IODimVjpAlgorithm(model, decay_or_rank=0.99)
    
    # 根据一个输入数据编译模型的eligibility trace，
    # 此后，调用该模型不仅更新模型的状态，还会更新模型的eligibility trace
    example_input = brainstate.random.random(10) < 0.1
    model.compile_graph(example_input)


本质上，用户定义的神经网络模型只是规定了模型状态 $\mathbf{h}$ 如何随着输入和时间前向演化，而在线学习算法 ``ETraceAlgorithm`` 编译后则定义了模型eligibility trace $\mathbf{\epsilon}$ 如何随着模型状态的更新而更新。这样，当我们再次调用模型时，不仅会更新模型的状态，还会更新模型的eligibility trace。

In [9]:
with brainstate.environ.context(dt=0.1 * u.ms):
    
    out = model(example_input)

# 通过调用 model.etrace_xs 可以获取模型对突触前神经活动追踪的 eligibility trace 
brainstate.util.PrettyMapping(model.etrace_xs)

{
  Var(id=1720127645760):float32[20]: ShortTermState(
    value=Array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0.], dtype=float32)
  )
}

In [10]:
# 通过调用 model.etrace_dfs 可以获取模型对突触后神经活动追踪的 eligibility trace
brainstate.util.PrettyMapping(model.etrace_dfs)

{
  (Var(id=1720127645824):float32[10], ('neu', 'V')): ShortTermState(
    value=Array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01],      dtype=float32)
  )
}

## 5. 总结

总的来说，`brainscale`为在线学习提供了一个完整而优雅的框架体系，其核心概念可以总结为以下几个层次：

1. **基础架构层**
   - 支持特定结构的动力学模型，将"动力学"和"交互"严格分离
   - 基于`brainstate`生态系统，完全兼容其编程范式
   - 通过`brainscale.nn`模块提供开箱即用的神经网络组件

2. **核心概念层**
   - `ETraceState`：标记需要进行eligibility trace更新的动力学状态
   - `ETraceOp`：定义动力学交互的具体操作
   - `ETraceParam`：组合参数和操作的统一接口

3. **算法实现层**
   - `DiagIODimAlgorithm`：基于ES-D-RTRL算法，具有$O(N)$复杂度
   - `DiagParamDimAlgorithm`：基于D-RTRL算法，具有$O(N^2)$复杂度
   - `DiagHybridDimAlgorithm`：混合算法，根据网络结构特点自适应选择$O(N)$或$O(N^2)$复杂度的算法

4. **工作流程**
   - 使用基础组件构建神经网络模型
   - 选择合适的在线学习算法封装模型
   - 编译模型生成eligibility trace计算图
   - 通过前向传播同时更新模型状态和eligibility trace

这个框架的独特之处在于：它将复杂的在线学习算法封装在简洁的接口后，提供灵活的定制化机制，既保持高性能，又确保易用性。同时，它与现有的 `brainstate` 生态系统无缝集成。通过这样的设计，`brainscale` 使构建和训练在线学习神经网络变得直观而高效，为神经计算研究提供了强大的工具。
